In [1]:
import tensorflow as tf
import chess.pgn
import pandas as pd
import numpy as np
import os

2025-04-18 20:21:24.621172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745029284.635540   32405 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745029284.639736   32405 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 20:21:24.655300: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

try:
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print('Now using gpu')
    else: print('Unable to use the gpu')


except RuntimeError as e:
    print(f'Something went wrong: {e}')


Now using gpu


In [3]:
file = '../data/raw/lichess_db_standard_rated_2016-03.pgn'

In [45]:
def chess_pipeline(pgn_file, max_games = None):
    data = []
    with open(pgn_file) as pgn:
        fmt_game = 0
        while True:

            game = chess.pgn.read_game(pgn)
            if game is None:
                print('No data found')
                break
            game_parsed = game_pipeline(game)
            data.append(game_parsed)
            fmt_game += 1
            print('Game(s) parsed:', fmt_game)
            if max_games is not None and fmt_game >= max_games:
                break
    return list(filter(None,data))



def game_pipeline(game):
    game_list = []
    board = game.board()
    result = game.headers['Result']
    outcome = 1 if result == '1-0' else -1 if result == '0-1' else 0
    moves = list(game.mainline_moves())
    if moves:
        for move in moves:
            fen = board.fen()
            move_uci = move.uci()
            move_san = board.san(move)
            player = 'white' if board.turn == chess.WHITE else 'black'

            game_list.append(
                {
                    'fen': fen,
                    'move_uci': move_uci,
                    'move_san': move_san,
                    'player': player,
                    'result': outcome
                })
            board.push(move)
    print('Game pipeline ended')
    return game_list

In [46]:
dataset = chess_pipeline(file,max_games = 3)

Game pipeline ended
Game(s) parsed: 1
Game pipeline ended
Game(s) parsed: 2
Game pipeline ended
Game(s) parsed: 3


In [48]:
dataset

[[{'fen': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
   'move_uci': 'e2e4',
   'move_san': 'e4',
   'player': 'white',
   'result': 1},
  {'fen': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
   'move_uci': 'e7e6',
   'move_san': 'e6',
   'player': 'black',
   'result': 1},
  {'fen': 'rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
   'move_uci': 'd2d3',
   'move_san': 'd3',
   'player': 'white',
   'result': 1},
  {'fen': 'rnbqkbnr/pppp1ppp/4p3/8/4P3/3P4/PPP2PPP/RNBQKBNR b KQkq - 0 2',
   'move_uci': 'd7d5',
   'move_san': 'd5',
   'player': 'black',
   'result': 1},
  {'fen': 'rnbqkbnr/ppp2ppp/4p3/3p4/4P3/3P4/PPP2PPP/RNBQKBNR w KQkq - 0 3',
   'move_uci': 'b1d2',
   'move_san': 'Nd2',
   'player': 'white',
   'result': 1},
  {'fen': 'rnbqkbnr/ppp2ppp/4p3/3p4/4P3/3P4/PPPN1PPP/R1BQKBNR b KQkq - 1 3',
   'move_uci': 'd5e4',
   'move_san': 'dxe4',
   'player': 'black',
   'result': 1},
  {'fen': 'rnbqkbnr/ppp2ppp/4p3/8/4p3/3P4/PPPN1PPP/R1BQ